<a href="https://colab.research.google.com/github/minyou2675/Bigdata_SchoolProject/blob/main/bagging%EB%AA%A8%EB%8D%B8_%EC%84%B1%EB%8A%A5%EB%B9%84%EA%B5%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
data = pd.read_csv('/content/kc_house_data.csv')

In [ ]:
data.head()

,id,date,price,bedrooms,bathrooms,floors,waterfront,condition,grade,yr_built,yr_renovated,zipcode,lat,long
0,7129300520,20141013T000000,221900.0,3,1.00,1.0,0,3,7,1955,0,98178,47.5112,-122.257
1,6414100192,20141209T000000,538000.0,3,2.25,2.0,0,3,7,1951,1991,98125,47.7210,-122.319
2,5631500400,20150225T000000,180000.0,2,1.00,1.0,0,3,6,1933,0,98028,47.7379,-122.233
3,2487200875,20141209T000000,604000.0,4,3.00,1.0,0,5,7,1965,0,98136,47.5208,-122.393
4,1954400510,20150218T000000,510000.0,3,2.00,1.0,0,3,8,1987,0,98074,47.6168,-122.045


In [ ]:
data.shape
n = data.shape[0]
m = data.shape[1]
print(n,m)

21613 14


In [ ]:
#data cleansing -> id, data, zipcode, latitude, longtitude

In [ ]:
data = data.drop(['id','date','zipcode','lat','long'], axis = 1)

In [ ]:
m = data.shape[1]
print (n,m)

21613 9


In [ ]:
#f(x) = y x는 속성-price y는 price
#x에서 train x test x 나누고 y에서 train y test y 나누기

In [ ]:
feature_columns = list(data.columns.difference(['price']))

In [ ]:
X = data[feature_columns]
y = data['price']
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [ ]:
len(train_x), len(test_x), len(train_y), len(test_y)

(15129, 6484, 15129, 6484)

In [ ]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(train_x, train_y)

train_pred = regressor.predict(train_x)
test_pred = regressor.predict(test_x)

print(train_pred, test_pred) #각각의 결과가 리스트에 나타남

from sklearn.metrics import mean_squared_error
from math import sqrt

train_mse = mean_squared_error(train_pred, train_y)
test_mse = mean_squared_error(test_pred, test_y)
print(sqrt(train_mse), sqrt(test_mse)) #test에 error가 더 많다

[ 718612.57288289  583162.20845483  580770.98393422 ...  308151.68024295
 -132165.16277028  297325.31617448] [ 562774.88835353  711422.76107068 1118395.8968444  ...  325841.31615172
  908826.64428964  434544.24125533]
230186.80162073454 239804.2967085815


linear regression 했을 때 test에서 에러는 239804가 나온다

In [ ]:
bagging_predict_result = []
for _ in range (10):
  data_index = [ i for i in range(train_x.shape[0])]
  #print(data_index)
  random_data_index = np.random.choice(data_index, train_x.shape[0])
  #print(random_data_index)
  #print(len(set(random_data_index)))
  #print(len(set(random_data_index))/ train_x.shape[0]) #전체 데이터의 63퍼 정도를 추출 나머지 37은 추출x
  sm_train_x = train_x.iloc[random_data_index,]
  sm_train_y = train_y.iloc[random_data_index,]

  regressor = LinearRegression()
  regressor.fit(sm_train_x, sm_train_y)
  sm_test_pred = regressor.predict(test_x)
  print(sqrt(mean_squared_error(sm_test_pred, test_y)))
  bagging_predict_result.append(sm_test_pred)


239958.4354042293
240052.60651663702
239816.7877902955
240133.2092333664
239704.15056713464
240038.852670593
240186.57988743132
240514.88232149128
239680.96074840488
240251.62572472304


In [ ]:
bagging_predict = [] #mse저장 변수
for i in range(test_x.shape[0]):
  temp_predict = []
  for j in range(len(bagging_predict_result)):
    temp_predict.append(bagging_predict_result[j][i])
  bagging_predict.append(np.mean(temp_predict)) #데이터를 평균내기

print(sqrt(mean_squared_error(bagging_predict, test_y)))

239746.34582940745


In [ ]:
from sklearn.ensemble import BaggingRegressor

regressor = LinearRegression()
bagging_model = BaggingRegressor(base_estimator = regressor, n_estimators = 5, verbose = 1)
#linear regressor를 5개 사용하는 bagging regressor 만듦
model = bagging_model.fit(train_x, train_y)
pred = model.predict(test_x)
print(sqrt(mean_squared_error(pred, test_y)))


239845.3351248124


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


에러가 239845으로 아주 살짝 증가하였다.

In [ ]:
bagging_model = BaggingRegressor(base_estimator = regressor, n_estimators = 30, verbose = 1)
#linear regressor를 5개 사용하는 bagging regressor 만듦
model = bagging_model.fit(train_x, train_y)
pred = model.predict(test_x)
print(sqrt(mean_squared_error(pred, test_y)))


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


239793.80986012478


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


30번으로 증가 시켰을 때 에러는 239793으로 아주 조금 개선되었다.\
bagging은 linear regression보단 \decision tree를 적용시키는게 더 적합하다


In [ ]:
from sklearn.tree import DecisionTreeRegressor #decision tree 불러옴

decision_tree = DecisionTreeRegressor()
model1 = decision_tree.fit(train_x, train_y)
pred1 = model1.predict(test_x)
print(sqrt(mean_squared_error(pred1, test_y)))

297446.8239488033


linear regrssion 보다 성능이 떨어지지만, default 모델이기 때문에, 파라미터를 개선하면 된다.

*Manual Bagging with Decision Tree

In [ ]:
bagging_predict_result = []
for _ in range (10):
  data_index = [ i for i in range(train_x.shape[0])]
  #print(data_index)
  random_data_index = np.random.choice(data_index, train_x.shape[0])
  #print(random_data_index)
  #print(len(set(random_data_index)))
  #print(len(set(random_data_index))/ train_x.shape[0]) #전체 데이터의 63퍼 정도를 추출 나머지 37은 추출x
  sm_train_x = train_x.iloc[random_data_index,]
  sm_train_y = train_y.iloc[random_data_index,]

  decision_tree = DecisionTreeRegressor()
  decision_tree.fit(sm_train_x, sm_train_y)
  sm_test_pred = decision_tree.predict(test_x)
  print(sqrt(mean_squared_error(sm_test_pred, test_y)))
  bagging_predict_result.append(sm_test_pred)

275034.74784007727
296262.703319783
303699.9691675843
296932.8618979542
312449.76505183155
306792.5588469935
285720.9665617889
286003.8156929117
298877.67585537094
288713.30437437334


In [ ]:
bagging_predict = [] #mse저장 변수
for i in range(test_x.shape[0]):
  temp_predict = []
  for j in range(len(bagging_predict_result)):
    temp_predict.append(bagging_predict_result[j][i])
  bagging_predict.append(np.mean(temp_predict)) #데이터를 평균내기

print(sqrt(mean_squared_error(bagging_predict, test_y)))

234328.20118182196


오류는 234328로 지금 나온 모델 중 가장 성능이 좋다.

In [ ]:
decision_tree = DecisionTreeRegressor()
bagging_model = BaggingRegressor(base_estimator =decision_tree, n_estimators =30, verbose = 1)
#DecisionTree regressor를 30번 사용하는 bagging regressor 만듦
model = bagging_model.fit(train_x, train_y)
pred = model.predict(test_x)
print(sqrt(mean_squared_error(pred, test_y)))


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


233741.29699006674


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


오토매틱 bagging 30회 결과 233741가 나온다. 여기서 가장 좋은 성능을 가진 모델

NameError: ignored